# IMPORTING AND LOADING

In [26]:
import subprocess
import torch
import torch.nn as nn
import time
from torch.autograd import Variable
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from scipy.linalg import norm

experiment = "seperate_models"

train_vectors = np.load("data/"+ experiment + "/train_vectors.npy")
train_sentences = np.load("data/"+ experiment + "/train_sentences.npy")
train_vectors = train_vectors / norm(train_vectors)

vec = train_vectors[::5], train_vectors[1::5], train_vectors[2::5], train_vectors[3::5], train_vectors[4::5]
sen = train_sentences[::5], train_sentences[1::5], train_sentences[2::5], train_sentences[3::5], train_sentences[4::5]

dataset1 = np.asarray(vec[0:2])
dataset2 = np.asarray(vec[0:3])
dataset3 = np.asarray(vec[0:4])
dataset4 = np.asarray(vec[0:5])

vec = None

sentences1 = np.asarray(sen[0:2])
sentences2 = np.asarray(sen[0:3])
sentences3 = np.asarray(sen[0:4])
sentences4 = np.asarray(sen[0:5])

sen = None


# MODEL DEFINITON AND PARAMETERS

In [27]:
class BasicGRU(nn.Module):
    def __init__(self, hidden_size, n_layers=1):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=0, bidirectional=True)       
        self.lin = nn.Linear(hidden_size, hidden_size)

    def forward(self, input_seq, input_lengths, hidden=None):

        packed = torch.nn.utils.rnn.pack_padded_sequence(input_seq, input_lengths, batch_first=True)

        outputs, hidden = self.gru(packed, hidden)
        
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs , batch_first=True)

        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]

        output = self.lin(outputs[:,-1,:].unsqueeze(1))
        return output

In [28]:
class StoryVectors(Dataset):

    def __init__(self, dataset, sentences):
        self.dataset = dataset
        self.type = self.dataset.shape[0]
        self.sen = sentences

        

    def __len__(self):
        return self.dataset.shape[1]

    def __getitem__(self, idx):
      
        if self.type == 2:
            X = [self.dataset[0][idx]]
            y = [self.dataset[1][idx]]
            sentences = [self.sen[0][idx], self.sen[1][idx]]
          
          
        elif self.type == 3:
            X = [self.dataset[0][idx], self.dataset[1][idx]]
            y = [self.dataset[2][idx]]
            sentences = [self.sen[0][idx], self.sen[1][idx], self.sen[2][idx]]
        
        elif self.type == 4:
            X = [self.dataset[0][idx], self.dataset[1][idx], self.dataset[2][idx]]
            y = [self.dataset[3][idx]]
            sentences = [self.sen[0][idx], self.sen[1][idx], self.sen[2][idx], self.sen[3][idx]]
        
        elif self.type == 5:
            X = [self.dataset[0][idx], self.dataset[1][idx], self.dataset[2][idx], self.dataset[3][idx]]
            y = [self.dataset[4][idx]]
            sentences = [self.sen[0][idx], self.sen[1][idx], self.sen[2][idx], self.sen[3][idx], self.sen[4][idx]]
        
        
        return [X, len(X), y, sentences]
      
def vocab_collate_func(batch):
    X = []
    y = []
    lengths = []
    sentences = []

    for datum in batch:
        X.append(datum[0])
        lengths.append(datum[1])
        y.append(datum[2])
        sentences.append(datum[3])

    return [torch.FloatTensor(X), torch.LongTensor(lengths), torch.FloatTensor(y), sentences]

In [29]:
model1 = torch.load("model/model1.tar", map_location={'cuda:0': 'cpu'})
model2 = torch.load("model/model2.tar", map_location={'cuda:0': 'cpu'})
model3 = torch.load("model/model3.tar", map_location={'cuda:0': 'cpu'})
model4 = torch.load("model/model4.tar", map_location={'cuda:0': 'cpu'})

### TEST LOADER

In [30]:
def nn(qvec, vectors, array, k=5):
    qvec = qvec / norm(qvec)
    print("computing scores")
    scores = np.dot(qvec, vectors.T).flatten()
    sorted_args = np.argsort(scores)[::-1]
    sentences = [(array[a], scores[a]) for a in sorted_args[:k]]
    for i, s in enumerate(sentences):
        print (s, sorted_args[i])
        
def suggestions(vectors, sentences, dataset_vectors, dataset_sentences, k=5):
    l, _ = vectors.shape
    vectors = np.append(vectors, np.zeros((1,4800)), axis=0)
    vectors = np.expand_dims(vectors, axis=1)
    sentences = np.append(sentences, "dummy sentence for label")
    sentences = np.expand_dims(sentences, axis=1)
    test_dataset = StoryVectors(vectors, sentences)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=1,
                                               collate_fn=vocab_collate_func,
                                               shuffle=True,
                                               num_workers=4)
    
    if l == 1:
        print("Calling model 1")
        model1.eval()
        for data, lengths, labels, sentences in test_loader:
                print("predicting vectors")
                pred = model1(data, lengths)
        pred = pred.detach().numpy().squeeze()
        nn(pred, dataset_vectors, dataset_sentences, k) 
        
    elif l == 2:
        print("Calling model 2")
        model2.eval()
        for data, lengths, labels, sentences in test_loader:
                print("predicting vectors")
                pred = model2(data, lengths)
        pred = pred.detach().numpy().squeeze()
        nn(pred, dataset_vectors, dataset_sentences, k)
        
    elif l == 3:
        print("Calling model 3")
        model3.eval()
        for data, lengths, labels, sentences in test_loader:
                print("predicting vectors")
                pred = model3(data, lengths)
        pred = pred.detach().numpy().squeeze()
        nn(pred, dataset_vectors, dataset_sentences, k)
        
    elif l == 4:
        print("Calling model 4")
        model4.eval()
        for data, lengths, labels, sentences in test_loader:
                print("predicting vectors")
                pred = model4(data, lengths)
        pred = pred.detach().numpy().squeeze()
        nn(pred, dataset_vectors, dataset_sentences, k)
        
    else: 
        print("Story too longgg.")
        return


# INTERFACE

How it works? 

This is an interactive story generating system that uses human authoring along with the system's expertise to generate intriguing short stories. The system generates the first sentence, or the story, following which it makes suggestions for the next sentence. You can choose from the, suggestions you see in the list by typing the corresponding digit for your choosen next sentence,  or you can eneter a sentence which then becomes the part of the narration. This procedure continues till we have a short 5 sentence story.

In [50]:
experiment = 'interface'
story = []
n = np.random.randint(0, len(sentences1[0]), 5)
print("INITIAL SENTENCE \n")
print('\n'.join([sentences1[0][i].decode('UTF-8') for i in n]))

INITIAL SENTENCE 

Emily decided to invite Emily's boyfriend over .
Emily was taking an English test .
John was trying to reheat some pizza .
John dreamed of one day owning a skateboard .
John loved stories about time travel .


In [51]:
s = input()
story.append(s)
print("Story so far: " + ' '.join(story))
l = len(story)
print("Converting to vectors... eta: 1 min")
p1 = subprocess.run(['/Users/shrey/anaconda3/envs/py2/bin/python', 'text2vec.py', ''.join(story)], stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
vectors = np.load("data/" + experiment + "/" + str(l) + ".npy")
suggestions(vectors, story, train_vectors, train_sentences, k=10)

 John hated airplanes.


Story so far: John hated airplanes.
Converting to vectors... eta: 1 min
Calling model 1
predicting vectors
computing scores
(b"John was sick of John's bandleader getting all the attention .", 0.0021618346) 102201
(b"John was having trouble controlling John's Great Dane .", 0.002161027) 96870
(b"John was worried sick about John's Dog .", 0.0021608546) 62509
(b"John was nervous about John's driving test at the DMV .", 0.0021597033) 108685
(b"John was driving John's humvee in the desert of Iraq .", 0.0021568637) 61730
(b"John was very confident in John's team .", 0.0021567023) 11296
(b"John was confident in John's ability .", 0.0021548271) 104687
(b"John was furious at John for breaking John's TV .", 0.0021542762) 11
(b"John was practicing John's equestrian ability .", 0.0021527433) 47305
(b"John was trained to be a pilot and did it John's whole military career .", 0.002152469) 103464


In [ ]:
s = input()
story.append(s)
print("Story so far: " + ' '.join(story))
l = len(story)
print("Converting to vectors... eta: 1 min")
p1 = subprocess.run(['/Users/shrey/anaconda3/envs/py2/bin/python', 'text2vec.py', ''.join(story)], stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
vectors = np.load("data/" + experiment + "/" + str(l) + ".npy")
suggestions(vectors, story, train_vectors, train_sentences, k=10)


 John took a train to Miami.


Story so far: John hated airplanes. John took a train to Miami.
Converting to vectors... eta: 1 min


In [44]:
s = input()
story.append(s)
print("Story so far: " + ' '.join(story))
l = len(story)
print("Converting to vectors... eta: 1 min")
p1 = subprocess.run(['/Users/shrey/anaconda3/envs/py2/bin/python', 'text2vec.py', ''.join(story)], stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
vectors = np.load("data/" + experiment + "/" + str(l) + ".npy")
suggestions(vectors, story, train_vectors, train_sentences, k=10)

 John is on John's way to the mountains .


Story so far: John is on a vacation with John's family . John is happy that John got to visit Cleveland . John is on John's way to the mountains .
Converting to vectors... eta: 1 min
Calling model 3
predicting vectors
computing scores
(b'When John reaches the campground John sees a sign .', 0.002080483) 11887
(b'Even though John is 50 John still screams when John sees one .', 0.0020136544) 64951
(b"Now John climbs onto John's rooftop to think quietly .", 0.0020058143) 87624
(b'John hits a hidden rock and ascends into the air .', 0.0019944683) 3138
(b'John notices John has a cavity .', 0.00199311) 84143
(b"John gets over John's fear and rides the airplane to see them .", 0.001988094) 83604
(b"John lowers John on one leg and twists John's body to enter the sedan .", 0.0019829988) 115082
(b'While walking John notices something on the side of the road .', 0.0019811895) 46421
(b"Suddenly John grabs John's chest .", 0.0019796751) 16512
(b'John decides John has to be one of the first to have 

In [45]:
s = input()
story.append(s)
print("Story so far: " + ' '.join(story))
l = len(story)
print("Converting to vectors")
p1 = subprocess.run(['/Users/shrey/anaconda3/envs/py2/bin/python', 'text2vec.py', ''.join(story)], stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
vectors = np.load("data/" + experiment + "/" + str(l) + ".npy")
suggestions(vectors, story, train_vectors, train_sentences, k=10)

 Suddenly John grabs John's chest .


Story so far: John is on a vacation with John's family . John is happy that John got to visit Cleveland . John is on John's way to the mountains . Suddenly John grabs John's chest .
Converting to vectors
Calling model 4
predicting vectors
computing scores
(b'John could feel John slipping .', 0.0018733903) 11233
(b'John was forced to vacate immediately .', 0.0018732201) 23294
(b'John kept having to turn John down .', 0.0018675218) 8853
(b'John was barely able to stop John .', 0.001866482) 61799
(b'John was glad John decided not to stop .', 0.0018607995) 109509
(b'John continued to let John go .', 0.0018602792) 108914
(b'John was able to quickly get to where John wanted .', 0.0018556078) 21123
(b'John had to stop to ask for directions .', 0.0018469059) 71118
(b'John was unable to stop in time .', 0.0018458194) 100638
(b'John was nervous John was going to fail .', 0.0018428373) 51817


## FINAL STORY

In [46]:
s = input()
story.append(s)
print("FINAL STORY: \n")
print("\n".join(story))

 John could feel John slipping .


FINAL STORY: 

John is on a vacation with John's family .
John is happy that John got to visit Cleveland .
John is on John's way to the mountains .
Suddenly John grabs John's chest .
John could feel John slipping .


In [25]:
s = input()
story.append(s)
print("FINAL STORY: \n")
print("\n".join(story))

 John decided to just leave it alone .


FINAL STORY: 

John was hanging out with friends .
They decided to split a pizza .
John ordered the pizza and waited for it to arrive .
There was one slice of pizza left .
John decided to just leave it alone .
